In [7]:
import pandas as pd
import iisignature
import os
import numpy as np

In [3]:
data = pd.read_csv("/Users/farhat/Documents/Project/ProcessedData/TrainingSet_B/p000006.psv", sep=',')
data['DBP']= data['DBP'].fillna((3*data['MAP']-data['SBP'])/2)
data.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,ShockIndex,BUN_CR,PartialSOFA,SepsisLabel
0,109.0,100.0,36.56,118.0,86.00,70.000,18.5,NaN,0.0,29.0,...,52.01,1,1,0,-0.03,3,0.923729,NaN,0.0,0
1,111.0,100.0,36.72,127.0,90.33,71.995,43.0,NaN,0.0,29.0,...,52.01,1,1,0,-0.03,4,0.874016,NaN,0.0,0
2,107.0,99.0,36.72,122.0,78.00,56.000,43.0,NaN,0.0,29.0,...,52.01,1,1,0,-0.03,5,0.877049,NaN,0.0,0
3,106.0,99.0,36.72,101.0,73.00,59.000,43.0,NaN,0.0,29.0,...,52.01,1,1,0,-0.03,6,1.049505,NaN,0.0,0
4,103.0,97.0,36.72,112.0,78.00,61.000,43.0,NaN,0.0,29.0,...,52.01,1,1,0,-0.03,7,0.919643,NaN,0.0,0


In [4]:
def compute_partial_sofa(df):
    # Coagulation (Platelets)
    df['SOFA_coagulation'] = pd.cut(df['Platelets'],
                                    bins=[-np.inf,20,50,100,150,np.inf],
                                    labels=[4,3,2,1,0]).astype(float)
    
    # Liver (Bilirubin)
    df['SOFA_liver'] = pd.cut(df['Bilirubin_direct'],
                              bins=[-np.inf,1.2,1.9,5.9,11.9,np.inf],
                              labels=[0,1,2,3,4]).astype(float)
    
    # Cardiovascular (MAP only, no vasopressors recorded)
    df['SOFA_cardiovascular'] = np.where(df['MAP'] >= 70, 0, 1)
    
    # Renal (Creatinine)
    df['SOFA_renal'] = pd.cut(df['Creatinine'],
                              bins=[-np.inf,1.2,1.9,3.4,4.9,np.inf],
                              labels=[0,1,2,3,4]).astype(float)
    
    # Partial SOFA = sum of available components
    df['PartialSOFA'] = df[['SOFA_coagulation','SOFA_liver','SOFA_cardiovascular','SOFA_renal']].sum(axis=1)
    df=df.drop(columns=['SOFA_coagulation','SOFA_liver','SOFA_cardiovascular','SOFA_renal'])
    
    return df


In [41]:
# Define source and destination folders
source_folder = '/Users/farhat/Documents/Project/RawData/TrainingSet_A'
destination_folder = '/Users/farhat/Documents/Project/ProcessedData/TrainingSet_A'

# Make sure destination folder exists
os.makedirs(destination_folder, exist_ok=True)

# Loop through each file in the source folder
for filename in os.listdir(source_folder):
    # Check if it is a file with the desired extension, e.g. .psv
    if filename.endswith('.psv'):
        # Construct full file path
        file_path = os.path.join(source_folder, filename)
        
        # Read the file into a DataFrame
        df = pd.read_csv(file_path, sep='|')
        
        # [Optional] Process your data here
        # For example, print shape or clean missing values
        df['DBP']= df['DBP'].fillna((3*df['MAP']-df['SBP'])/2) #Filling DBP calculating the value from SBP and MAP. 
        df=df.ffill()
        df=df.bfill()

        #Hand Crafted Data
        df['ShockIndex'] = df['HR'] / df['SBP']
        df['BUN_CR'] = df['Bilirubin_direct'] / df['Creatinine']
        # Partial SOFA and SOFA deterioration require definitions based on available variables
        # Placeholder below:
        df=compute_partial_sofa(df)
        col = df.pop('SepsisLabel')
        df['SepsisLabel']=col
        
        # Save to destination folder with the same filename
        save_path = os.path.join(destination_folder, filename)
        df.to_csv(save_path, index=False)

        print(f'Saved to {save_path}')


Saved to /Users/farhat/Documents/Project/ProcessedData/TrainingSet_A/p014977.psv
Saved to /Users/farhat/Documents/Project/ProcessedData/TrainingSet_A/p000902.psv
Saved to /Users/farhat/Documents/Project/ProcessedData/TrainingSet_A/p009098.psv
Saved to /Users/farhat/Documents/Project/ProcessedData/TrainingSet_A/p008386.psv
Saved to /Users/farhat/Documents/Project/ProcessedData/TrainingSet_A/p018195.psv
Saved to /Users/farhat/Documents/Project/ProcessedData/TrainingSet_A/p009926.psv
Saved to /Users/farhat/Documents/Project/ProcessedData/TrainingSet_A/p016806.psv
Saved to /Users/farhat/Documents/Project/ProcessedData/TrainingSet_A/p002873.psv
Saved to /Users/farhat/Documents/Project/ProcessedData/TrainingSet_A/p002867.psv
Saved to /Users/farhat/Documents/Project/ProcessedData/TrainingSet_A/p009932.psv
Saved to /Users/farhat/Documents/Project/ProcessedData/TrainingSet_A/p020378.psv
Saved to /Users/farhat/Documents/Project/ProcessedData/TrainingSet_A/p016812.psv
Saved to /Users/farhat/Docum

In [45]:
df=pd.read_csv("/Users/farhat/Documents/Project/ProcessedData/TrainingSet_B/p000006.psv", sep=',')
data=pd.DataFrame(columns=df.columns)
source_folder = '/Users/farhat/Documents/Project/ProcessedData/TrainingSet_A'
for filename in os.listdir(source_folder):
    # Check if it is a file with the desired extension, e.g. .psv
    if filename.endswith('.psv'):
        # Construct full file path
        file_path = os.path.join(source_folder, filename)
        
        # Read the file into a DataFrame
        df = pd.read_csv(file_path, sep=',')
        data = pd.concat([data, df], ignore_index=True)

/var/folders/zc/cx2984q17xs9k62vy4ms148w0000gn/T/ipykernel_62661/1333204208.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, df], ignore_index=True)


In [5]:
data.head(10)

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,ShockIndex,BUN_CR,PartialSOFA,SepsisLabel
0,109.0,100.0,36.56,118.0,86.00,70.000,18.5,NaN,0.0,29.0,...,52.01,1,1,0,-0.03,3,0.923729,NaN,0.0,0
1,111.0,100.0,36.72,127.0,90.33,71.995,43.0,NaN,0.0,29.0,...,52.01,1,1,0,-0.03,4,0.874016,NaN,0.0,0
2,107.0,99.0,36.72,122.0,78.00,56.000,43.0,NaN,0.0,29.0,...,52.01,1,1,0,-0.03,5,0.877049,NaN,0.0,0
3,106.0,99.0,36.72,101.0,73.00,59.000,43.0,NaN,0.0,29.0,...,52.01,1,1,0,-0.03,6,1.049505,NaN,0.0,0
4,103.0,97.0,36.72,112.0,78.00,61.000,43.0,NaN,0.0,29.0,...,52.01,1,1,0,-0.03,7,0.919643,NaN,0.0,0
5,100.0,99.0,36.67,122.0,85.33,66.995,43.0,NaN,0.0,29.0,...,52.01,1,1,0,-0.03,8,0.819672,NaN,0.0,0
6,96.0,98.0,36.67,111.0,75.67,58.005,43.0,NaN,0.0,29.0,...,52.01,1,1,0,-0.03,9,0.864865,NaN,0.0,0
7,95.0,98.0,36.67,129.0,95.00,78.000,43.0,NaN,0.0,29.0,...,52.01,1,1,0,-0.03,10,0.736434,NaN,0.0,0
8,94.0,99.0,36.67,144.0,100.00,78.000,43.0,NaN,0.0,29.0,...,52.01,1,1,0,-0.03,11,0.652778,NaN,0.0,0
9,95.0,98.0,36.72,115.0,89.00,76.000,43.0,NaN,0.0,29.0,...,52.01,1,1,0,-0.03,12,0.826087,NaN,0.0,0


In [6]:
data.shape

(17, 44)

In [51]:
data.to_csv("/Users/farhat/Documents/Project/ProcessedData/fullData.csv", index=False)